### Data Science Capstone : Week 3 Assignment - Part 1 and Part 2

## Part 1

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# Extracting from wiki url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')

In [3]:
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

In [4]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)
df[df.duplicated(['PostalCode'], keep=False)]

(20, 3)


,PostalCode,Borough,Neighborhood


In [5]:
df.head(2)

,PostalCode,Borough,Neighborhood
0,\nM1ANot assigned,\nM2ANot assigned,\nM3ANorth York(Parkwoods)
1,\nM1BScarborough(Malvern / Rouge),\nM2BNot assigned,\nM3BNorth York(Don Mills)North


In [6]:
#Webscrapping tips from the instructions (Split, strip, replace and append)
table_contents=[] 
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [7]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                     'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                     'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [8]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [9]:
df.shape

(103, 3)

In [10]:
#End of part 1 of assignment

## Part 2

In [11]:
dfgeo = pd.read_csv("https://cocl.us/Geospatial_data") #using link to access the csv file
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
dfgeo.shape

(103, 3)

In [12]:
df.head(2)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village


In [13]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')
df2.head(2)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572


In [14]:
df2.shape

(103, 5)